In [1]:
import pandas as pd
df = pd.read_csv("RaceTimes.csv")

In [4]:
import pandas as pd
import re
from datetime import datetime

df = pd.read_csv("RaceTimes.csv")

def convert_to_seconds(time_str):
    minutes, seconds, milliseconds = [int(part) for part in time_str.replace('.', ':').split(':')]
    return minutes * 60 + seconds + milliseconds / 1000

def convert_id_readable(race_id):
    # Use regular expressions to extract the time and date parts
    match = re.match(r"(\d+):(\d+)(\d{2})\.(\d{2}\.\d{4})", race_id)
    
    if not match:
        return "Invalid format"

    # Extract the hour, minute, day, and the rest of the date parts
    hour, minute, day, month_year = match.groups()
    
    # Format the hour to 12-hour time and assume it is PM
    hour_int = int(hour)
    # If the hour is less than 12, we assume it's PM, otherwise, we convert to 12-hour format
    hour_formatted = f"{hour_int if hour_int <= 12 else hour_int - 12}"
    time_suffix = "pm" if hour_int <= 12 else "am"
    
    # Construct the time with the correct suffix
    time = f"{hour_formatted}:{minute}{time_suffix}"
    
    # Construct the date in the desired format
    date_formatted = datetime.strptime(f"{day}.{month_year}", "%d.%m.%Y").strftime("%d/%m/%Y")
    
    # Return the formatted string
    return f"{time} @ {date_formatted}"

print(df)


    Time of Race        Date             Racer  Lap   Lap Time Position   
0           5:32  06.01.2024  mitchell.whitten    1  01:17.050        #  \
1           5:32  06.01.2024  mitchell.whitten    2  00:37.893        #   
2           5:32  06.01.2024  mitchell.whitten    3  00:36.681        #   
3           5:32  06.01.2024  mitchell.whitten    4  00:37.669        #   
4           5:32  06.01.2024  mitchell.whitten    5  00:35.939        #   
..           ...         ...               ...  ...        ...      ...   
205         5:12  16.11.2023       darian.king   10  00:38.893      NaN   
206         5:12  16.11.2023       darian.king   11  00:40.677      NaN   
207         5:12  16.11.2023       darian.king   12  00:39.268      NaN   
208         5:12  16.11.2023       darian.king   13  00:38.254      NaN   
209         5:12  16.11.2023       darian.king   14  00:40.071      NaN   

             RaceID  Lap Time Seconds          RaceID Name  
0    5:3206.01.2024            77.050 

In [6]:
df['Lap Time Seconds'] = df['Lap Time'].apply(convert_to_seconds)
df['RaceID Name'] = df['RaceID'].apply(convert_id_readable)

# Split 'RaceID Name' into two columns 'Race Date' and 'Race Time'
df['Race Date'], df['Race Time'] = df['RaceID Name'].str.split('@',n=1,expand=True)

# Convert 'Race Date' to a sortable format
df['Race Date'] = pd.to_datetime(df['Race Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

# Sort by 'Race Date' then 'Race Time'
df = df.sort_values(by=['Race Date', 'Race Time'])


print(df)